Connect to Azure Open AI API 

In [ ]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_type = "azure"
openai.api_version = "2023-07-01-preview" 
openai.api_base = os.getenv('OPENAI_API_BASE')  
openai.api_key = os.getenv('OPENAI_API_KEY') 

Doing some sample calls to ChatGPT

In [ ]:
response = openai.ChatCompletion.create(
    engine="qpt-4-32k",  # "gpt-35-turbo" "qpt-4-32k" 
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Tell me a funny joke about SharePoint"}
    ]
)
print(response['choices'][0]['message']['content'])

Asking ChatGPT to summarize a document

In [ ]:
import docx
from docx import Document
import sys

def summarize_text(text_to_summarize):
    response = openai.ChatCompletion.create(
        engine="qpt-4-32k",  # "gpt-35-turbo" "qpt-4-32k" 
        messages=[
            {"role": "system", "content": "Assistant summarize a meeting transcript with a reading time of 30 seconds"},
            {"role": "user", "content": text_to_summarize}
        ]
    )
    return response['choices'][0]['message']['content']

def summarize_meeting_transcript(input_file):
    doc = docx.Document(input_file)
    all_paras = doc.paragraphs

    summary = Document()

    meeting_transcript = ""
    for para in all_paras:
        lines = para.text.split("\n")
        if len(lines) == 3:
            # For meeting transcript
            speaker, speech = lines[1], lines[2]
            meeting_transcript += f"\n{speaker}: {speech}"
        else:
            meeting_transcript += f"\n{para.text}"

    chunk_size = 100000
    for i in range(0, len(meeting_transcript), chunk_size):
        chunk = meeting_transcript[i:i+chunk_size]
        print(f"Working on chunk {i//chunk_size+1} of {len(meeting_transcript)//chunk_size+1}")
        summary.add_paragraph(summarize_text(chunk))

    summary_file = input_file.split(".")[0] + " in 30 seconds.docx"
    summary.save(summary_file)
    print(f"\nFinished summarizing meeting transcript. Summary saved to {summary_file}")

summarize_meeting_transcript("Archidevs_Exchange_2023-02-20.docx")